In [ ]:
import boto3
import os
import pandas as pd
import yfinance as yf
import json
from datetime import datetime
from dotenv import load_dotenv

Configurations

In [ ]:
load_dotenv("enviroment/.env")

aws_access_key = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_key = os.getenv("AWS_SECRET_ACCESS_KEY")
aws_region = os.getenv("AWS_REGION")
bucket_name = "silver-finance-data"

In [ ]:
s3_client = boto3.client(
    "s3",
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
    region_name=os.getenv("AWS_REGION"),
)
print(s3_client)

Functions

In [ ]:
def put_json_yfinance_aws(s3_client, ticker, ticker_label, start_date, end_date,bucket_name):
    data = yf.download(ticker, start=start_date, end=end_date)
    data.columns = data.columns.get_level_values(0)
    data = data.copy()
    data = data.reset_index(drop=False)
    for index, rows in data.iterrows():
        date = rows['Date']
        close = round(rows['Close'],3)
        high = round(rows['High'],3)
        low = round(rows['High'],2)
        open = round(rows['Open'],2)
        volume = round(rows['Volume'],2)
        formatted_key_date = date.strftime("%Y_%m_%d")
        formatted_date = date.strftime("%Y-%m-%d")
        json_data = {"date_capture" : formatted_date,"close" : close,"high" : high,"low" : low,"open" : open,"volume":volume}
        json_data = json.dumps(json_data)
        s3_object_name = ticker_label + "/" + formatted_key_date + "_" + ticker_label
        s3_client.put_object(
            Bucket = bucket_name,
            Key = s3_object_name,
            Body=json_data,
            ContentType="application/json"
            )
        print(s3_object_name)


    return data

In [ ]:
for itens in ['ITUB3.SA','ABEV3.SA','BBDC3.SA','PETR3.SA','^BVSP']:
    ticker = itens
    ticker_label = ticker.replace('^','')
    ticker_label = ticker_label.replace('.SA','')
    start_date = '1900-01-01'
    end_date = '2024-04-26'
    put_json_yfinance_aws(s3_client,ticker, ticker_label, start_date, end_date, 'silver-finance-data')